In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle 
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
import math
# from tensorflow.python.client import device_lib601

/home/praphul/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from numpy import random

In [3]:
with open('embedding_p2', 'rb') as f:
    word_embedding = pickle.load(f)

with open('vocab_p2', 'rb') as f:
    vocab = pickle.load(f)


word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

# with open("data_final", 'rb') as f:
#     full_data = pickle.load(f)

In [4]:
# random.shuffle(full_data)

In [5]:
# full_data = full_data[:19000]

In [6]:
# full_data[28][2]

In [7]:
mode = 'train'
num_classes = 2
batch_size = 1
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_para_size = 250
max_title_size = 22
hidden_size = 100
hidden_size2 = 50
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.001
decay_steps = 5000
decay_rate = 0.96
clip_norm = 1
clipping = True

In [8]:
def word_emb_maker(mode, x, word_emb_mat, shape_arr):
    with tf.name_scope("word_embedding_layer"):
        if mode == 'train':
            unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
        else:
            unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')

        final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
        Wx = tf.nn.embedding_lookup(final_word_emb_mat, x)
        
        return Wx

In [9]:
def word_emb_maker_title(mode, x, word_emb_mat, shape_arr):
    with tf.name_scope("word_embedding_layer_top"):
        if mode == 'train':
            unk_word_emb_mat_t = tf.get_variable("word_emb_mat_t", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
        else:
            unk_word_emb_mat_t = tf.get_variable("word_emb_mat_t", shape=[unk_vocab_size, word_emb_size], dtype='float')

        final_word_emb_mat_t = tf.concat([word_emb_mat, unk_word_emb_mat_t], 0)
        Wx_t = tf.nn.embedding_lookup(final_word_emb_mat_t, x)
        
        return Wx_t

In [10]:
x = tf.placeholder('int32', [batch_size, max_para_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
word_emb_mat = tf.placeholder('float', [vocab_size, word_emb_size], name='emb_mat')

x_mask = tf.placeholder('bool', [batch_size, max_para_size], name="x_mask")    
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 1)    

 #title
x_title = tf.placeholder('int32', [batch_size, max_title_size], name="x_title")
x_title_mask  = tf.placeholder('bool', [batch_size, max_title_size], name='x_title_mask')
x_title_len = tf.reduce_sum(tf.cast(x_title_mask, 'int32'), 1)

x_title_total = tf.placeholder('int32', [None, max_title_size+1], name="x_title_total")
x_title_total_mask  = tf.placeholder('bool', [None, max_title_size+1], name='x_title_total_mask')
x_title_total_len = tf.reduce_sum(tf.cast(x_title_total_mask, 'int32'), 1)
 

is_train = tf.placeholder('bool', [], name='is_train')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))


Wx = word_emb_maker(mode='train', x=x, word_emb_mat=word_emb_mat, shape_arr=[unk_vocab_size, word_emb_size])

Wx_title= word_emb_maker_title(mode='train',x=x_title, word_emb_mat=word_emb_mat, shape_arr=[unk_vocab_size, word_emb_size])

with tf.variable_scope("lstm1"):
    cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size2,state_is_tuple=True)
    cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size2,state_is_tuple=True)

    d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
    d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)
    
    (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
    h = tf.concat([fw_h1, bw_h1], 2)


def attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

with tf.variable_scope("attention"):
    W_att = tf.Variable(tf.truncated_normal([hidden_size, 1], mean=0.0, stddev=0.1, seed=0), name="W_att")
    c = tf.expand_dims(attention(h[0], x_mask[0], W_att), 0)
    for i in range(1, batch_size):
        c = tf.concat([c, tf.expand_dims(attention(h[i], x_mask[i], W_att), 0)], 0)


In [11]:
c

<tf.Tensor 'attention/ExpandDims:0' shape=(1, 100) dtype=float32>

In [12]:
Wx_final = tf.concat([tf.expand_dims(c,1), Wx_title],1)

In [13]:
Wx_final

<tf.Tensor 'concat:0' shape=(1, 23, 100) dtype=float32>

In [14]:
with tf.variable_scope("lstm_layer_top"):
    cell_fw_t = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw_t = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw_t = tf.contrib.rnn.DropoutWrapper(cell_fw_t, input_keep_prob=input_keep_prob)
    d_cell_bw_t = tf.contrib.rnn.DropoutWrapper(cell_bw_t, input_keep_prob=input_keep_prob)
    
    (fw_h_t, bw_h_t), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw_t, d_cell_bw_t, Wx_final, sequence_length=x_title_len, dtype='float', scope='lstm_top')
    ht = tf.concat([fw_h_t, bw_h_t], 2)

In [15]:
ht

<tf.Tensor 'lstm_layer_top/concat:0' shape=(1, 23, 200) dtype=float32>

In [16]:
with tf.variable_scope("W_att_local_top"):
    W_att_local_t = tf.get_variable("W_att_local_t", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
    flat_h_t = tf.reshape(ht, [-1, 2*hidden_size])
    h_tanh_t = tf.tanh(flat_h_t)
    u_flat_t = tf.matmul(h_tanh_t, W_att_local_t)

    u_local_t = tf.reshape(u_flat_t, [-1, max_title_size+1])
    final_u_t = (tf.cast(x_title_total_mask, 'float')-1)*(1e7) + u_local_t

    ct = tf.reduce_sum(tf.multiply(ht, tf.expand_dims(tf.nn.softmax(final_u_t, dim=-1), 2)), 1)

In [17]:
ct

<tf.Tensor 'W_att_local_top/Sum:0' shape=(?, 200) dtype=float32>

In [18]:
with tf.variable_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_classes], mean=0.0, stddev=0.1, seed=0), name="W")
    b = tf.Variable(tf.zeros([num_classes]), name="b")
    drop_c = tf.nn.dropout(ct, input_keep_prob)
    logits = tf.matmul(drop_c, W) + b
    predictions = tf.argmax(logits, 1)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
global_step = tf.Variable(0, trainable=False, name="global_step")


learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

tv_all = tf.trainable_variables()
tv_regu =[]
for t in tv_all:
    if t.name.find('b:')==-1:
        tv_regu.append(t)

l2_loss = l2_lambda * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])

total_loss = loss + l2_loss

optimizer = tf.train.AdamOptimizer(learning_rate)


grads_vars = optimizer.compute_gradients(total_loss)

clipped_grads = grads_vars
if(clipping == True):
    clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

train_op = optimizer.apply_gradients(clipped_grads, global_step=global_step)

var_summaries = []
for v in tv_all:
    var_summary = tf.summary.histogram("{}/var".format(v.name), v)
    var_summaries.append(var_summary)

var_summaries_merged = tf.summary.merge(var_summaries)

loss_summary = tf.summary.scalar("loss", loss)
total_loss_summary = tf.summary.scalar("total_loss", total_loss)
summary = tf.summary.merge_all()

In [19]:
model_name = 'new'
model_dir = 'output/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(model_dir):
    print(1)
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [20]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# print (device_lib.list_local_devices())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
saver = tf.train.Saver()                                             # For Saving the model
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [21]:
# #Splitting
# data_x = [data[0:2] for data in full_data]
# data_y = [data[2] for data in full_data]
# x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, train_size=0.8, test_size=0.2, shuffle=True, stratify=data_y, random_state=0)

# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.9, test_size=0.1, shuffle=True, stratify=y_train, random_state=0)

In [22]:
def data_prepare(x):
    #x=[news[1] for news in x_]
    num_examples = len(x)

    xx = np.zeros([num_examples, max_para_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_para_size], dtype=bool)
    
    xx_title = np.zeros([num_examples, max_title_size], dtype=int)
    xx_title_mask = np.zeros([num_examples, max_title_size], dtype=bool)
    
    xx_total_title = np.zeros([num_examples, max_title_size+1], dtype=int)
    xx_total_title_mask = np.zeros([num_examples, max_title_size+1], dtype=bool)
      

    for j in range(num_examples):
        for k in range(max_para_size):
            w = x[j][1][k]
            if w == 'PAD':
                xx[j][k] = word2id[w]
            elif w in word2id:
                xx[j][k] = word2id[w]
                xx_mask[j][k] = True
            else:
                xx[j][k] = word2id['UNKNOWN_TOKEN']
                
    
    for j in range(num_examples):
        for i in range(max_title_size):
            w = x[j][0][i]
            if w == 'PAD':
                xx_title[j][i] = word2id[w]
            elif w in word2id:
                xx_title[j][i] = word2id[w]
                xx_title_mask[j][i] = True
            else:
                xx_title[j][i] = word2id['UNKNOWN_TOKEN']
    
    for j in range(num_examples):
        xx_total_title_mask[j][0] = True
        for i in range(max_title_size):
            w = x[j][0][i]
            if w == 'PAD':
                pass
            elif w in word2id:
                xx_total_title_mask[j][i+1] = True

                
            
    return xx, xx_mask ,xx_title,xx_title_mask,xx_total_title,xx_total_title_mask

def eval_score(yy, pred):
    num_batches = int(len(yy)/batch_size)
    f1 = f1_score(yy[:batch_size*num_batches], pred, average='macro')
    accu = accuracy_score(yy[:batch_size*num_batches], pred)
    return f1*100, accu*100

def model(xx, yy, xx_mask, xx_title, xx_title_mask, xx_title_total_mask, train_cond=True):
    num_batches = int(len(xx)/batch_size)
    losses = 0
    preds = []
    for j in range(num_batches): 
        
        s = j * batch_size
        e = (j+1) * batch_size
        
        feed_dict = {word_emb_mat:word_embedding,
                     input_keep_prob:keep_prob,
                     is_train:train_cond,
                     x:xx[s:e], y:yy[s:e], x_mask:xx_mask[s:e],x_title:xx_title[s:e],
                     x_title_mask:xx_title_mask[s:e],
                     x_title_total_mask:xx_title_total_mask[s:e]}
        
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, total_loss, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)  
#             print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred = sess.run([total_loss, predictions], feed_dict)
            preds.append(pred)
            
        losses +=_loss

    if(train_cond==False):
        y_pred = []
        for i in range(num_batches):
            for pred in preds[i]:
                y_pred.append(pred)
        return losses/num_batches, y_pred
    
    return losses/num_batches, step

In [23]:
# xx_train, xx_mask_train ,xx_title_train,xx_title_mask_train,xx_total_title_train,xx_total_title_mask_train = data_prepare(x_train)
# xx_val, xx_mask_val ,xx_title_val,xx_title_mask_val,xx_total_title_val,xx_total_title_mask_val= data_prepare(x_val)
# xx_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_test,xx_total_title_mask_test = data_prepare(x_test)
# yy_train = np.array(y_train).reshape(-1)
# yy_val = np.array(y_val).reshape(-1)
# yy_test = np.array(y_test).reshape(-1)

In [25]:
import pandas as pd
import jsonlines
import numpy as np
import random
import pickle as pkl
import re
import sklearn 
import pickle as pkl
import glob
from nltk import word_tokenize   
import nltk

In [27]:
def filter(line, whitelist):
    ##############################################################
    #   Filters out all characters which are not in whitelist    #
    ##############################################################

    return ''.join([ch for ch in line if ch in whitelist])


WHITELIST1 = "0123456789abcdefghijklmnopqrstuvwxyz' "
WHITELIST2 = "0123456789abcdefghijklmnopqrstuvwxyz' "

In [28]:
def preprocess(string): 
    string = re.sub("\.\.\.", " \. ", string)
    string = re.sub(r"won't", "will not", string)
    string = re.sub(r"n\'t", " not", string)
    string = re.sub(r"n't", " not ", string)
    string = re.sub(r"\'re", " are", string)
    string = re.sub(r"\'s", " is", string)
    string = re.sub(r"\'d", " would", string)
    string = re.sub(r"\'ll", " will", string)
    string = re.sub(r"\'t", " not", string)
    string = re.sub(r"\'ve", " have", string)
    string = re.sub(r"\'m", " am", string)
    string = re.sub(r"\'d've", " would have", string)
    string = re.sub(r"\'d'y", " do you", string)
    string = re.sub(r"'", " ", string)
    return string

In [29]:
def process(str1, str2):
    str1 = filter(str1.lower(), WHITELIST2)
    str2 = filter(str2.lower(), WHITELIST1)
    str1 = preprocess(str1)
    str2 = preprocess(str2)
    str1 = str1.split()
    str2 = str2.split()
    test = []
    temp1 = []
    temp2 = []
    for i in range(22):
        if(i<len(str1)):
            temp1.append(str1[i])
        else:
            temp1.append('PAD')
    for i in range(250):
        if(i<len(str2)):
            temp2.append(str2[i])
        else:
            temp2.append('PAD')
            
    test.append(np.array([temp1, temp2]))
    return test

In [79]:
f = open('heading4.txt','r')
str1 = f.read()
f.close()

f = open('article4.txt', 'r')
str2 = f.read()
f.close()

In [80]:
test = process(str1, str2)

In [81]:
t_test, t_mask_test ,t_title_test,t_title_mask_test,t_total_title_test,t_total_title_mask_test = data_prepare(test)


In [82]:
t_test_yy = np.zeros([len(t_test), 1])
t_test_yy = np.array(t_test_yy).reshape(-1)

In [83]:
saver.restore(sess, save_dir)

In [84]:
feed_dict = {word_emb_mat:word_embedding,
             input_keep_prob:keep_prob,
             is_train:False,
             x:t_test, y:t_test_yy, x_mask:t_mask_test,x_title:t_title_test,
             x_title_mask:t_title_mask_test,
             x_title_total_mask:t_total_title_mask_test}

In [85]:
pred = sess.run([predictions], feed_dict)

In [86]:
if(pred[0][0]==0):
    print("AUTHENTIC")
else:
    print("FAKE")

FAKE


In [23]:
# xx_train.shape, xx_val.shape, xx_test.shape

((23759, 250), (2640, 250), (6600, 250))

In [24]:
# import time

In [24]:
# num_epochs = 1
# for i in range(num_epochs):
#     start_time = time.time()
#     random = np.random.choice(len(yy_train), size=(len(yy_train)), replace=False)
#     xx_train = xx_train[random]
#     yy_train = yy_train[random]
#     xx_mask_train = xx_mask_train[random]
#     xx_title_train = xx_title_train[random]
#     xx_title_mask_train = xx_title_mask_train[random]                           
#     xx_total_title_train = xx_total_title_train[random]
#     xx_total_title_mask_train = xx_total_title_mask_train[random]
            
#     losses, step = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train
#                         ,xx_total_title_mask_train, train_cond=True)
    
#     print("Epoch:", i+1,"Step:", step, "loss:",losses, "Epoch_Time:",time.time() - start_time)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")

#     val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                                xx_total_title_mask_val, train_cond=False)

#     f1_, accu_ = eval_score(y_val, val_pred)
#     print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
# test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_mask_test, train_cond=False)

# f1_, accu_ = eval_score(y_test, test_pred)
# print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Epoch: 1 Step: 57 loss: 1.1143866066347088 Epoch_Time: 656.621645450592
Saved Model Complete
Val: F1 Score:  49.19813797957208 Accuracy:  60.5 Loss:  1.0870561798413594
Test: F1 Score:  50.48662740126984 Accuracy:  61.68749999999999 Loss:  1.0792090892791748


In [35]:
# num_epochs = 5
# for i in range(num_epochs):
#     start_time = time.time()
#     random = np.random.choice(len(yy_train), size=(len(yy_train)), replace=False)
#     xx_train = xx_train[random]
#     yy_train = yy_train[random]
#     xx_mask_train = xx_mask_train[random]
#     xx_title_train = xx_title_train[random]
#     xx_title_mask_train = xx_title_mask_train[random]                           
#     xx_total_title_train = xx_total_title_train[random]
#     xx_total_title_mask_train = xx_total_title_mask_train[random]
            
#     losses, step = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train
#                         ,xx_total_title_mask_train, train_cond=True)
    
#     print("Epoch:", i+1,"Step:", step, "loss:",losses, "Epoch_Time:",time.time() - start_time)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")

#     val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                                xx_total_title_mask_val, train_cond=False)

#     f1_, accu_ = eval_score(y_val, val_pred)
#     print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
#     test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_mask_test, train_cond=False)

#     f1_, accu_ = eval_score(y_test, test_pred)
#     print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Epoch: 1 Step: 119 loss: 1.0566360009344 Epoch_Time: 713.6098966598511
Saved Model Complete
Val: F1 Score:  53.97080614142943 Accuracy:  63.5 Loss:  1.032202164332072
Test: F1 Score:  56.07968177306793 Accuracy:  64.875 Loss:  1.0312700867652893
Epoch: 2 Step: 176 loss: 1.0116695186548066 Epoch_Time: 688.9951281547546
Saved Model Complete
Val: F1 Score:  60.73082010582011 Accuracy:  68.33333333333333 Loss:  0.9778481324513754
Test: F1 Score:  62.61618331747181 Accuracy:  69.25 Loss:  0.982907809317112
Epoch: 3 Step: 233 loss: 0.9619972078423751 Epoch_Time: 702.7048165798187
Saved Model Complete
Val: F1 Score:  64.42754513627827 Accuracy:  70.16666666666667 Loss:  0.9442667961120605
Test: F1 Score:  65.24805854818612 Accuracy:  70.5 Loss:  0.944663755595684
Epoch: 4 Step: 290 loss: 0.915286360079782 Epoch_Time: 959.0774850845337
Saved Model Complete
Val: F1 Score:  67.85839944972565 Accuracy:  71.83333333333334 Loss:  0.8972695171833038
Test: F1 Score:  68.3096038523638 Accuracy:  72.0 

In [26]:
# num_epochs = 5
# for i in range(num_epochs):
#     start_time = time.time()
#     random = np.random.choice(len(yy_train), size=(len(yy_train)), replace=False)
#     xx_train = xx_train[random]
#     yy_train = yy_train[random]
#     xx_mask_train = xx_mask_train[random]
#     xx_title_train = xx_title_train[random]
#     xx_title_mask_train = xx_title_mask_train[random]                           
#     xx_total_title_train = xx_total_title_train[random]
#     xx_total_title_mask_train = xx_total_title_mask_train[random]
            
#     losses, step = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train
#                         ,xx_total_title_mask_train, train_cond=True)
    
#     print("Epoch:", i+1,"Step:", step, "loss:",losses, "Epoch_Time:",time.time() - start_time)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")

#     val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                                xx_total_title_mask_val, train_cond=False)

#     f1_, accu_ = eval_score(y_val, val_pred)
#     print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
#     test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_mask_test, train_cond=False)

#     f1_, accu_ = eval_score(y_test, test_pred)
#     print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Epoch: 1 Step: 433 loss: 0.7532537614190301 Epoch_Time: 1204.6124732494354
Saved Model Complete
Val: F1 Score:  73.64769385494773 Accuracy:  77.0 Loss:  0.7106629543834262
Test: F1 Score:  73.97625173511588 Accuracy:  77.33333333333333 Loss:  0.7215534076094627
Epoch: 2 Step: 519 loss: 0.648054113221723 Epoch_Time: 1256.3568346500397
Saved Model Complete
Val: F1 Score:  81.49071645847643 Accuracy:  83.0 Loss:  0.5837695863511827
Test: F1 Score:  80.01040554232044 Accuracy:  81.75 Loss:  0.6172010749578476
Epoch: 3 Step: 605 loss: 0.5910185152708098 Epoch_Time: 1202.3736617565155
Saved Model Complete
Val: F1 Score:  81.01400947520384 Accuracy:  81.55555555555556 Loss:  0.5741049514876472
Test: F1 Score:  81.62673990631573 Accuracy:  82.20833333333334 Loss:  0.5841019079089165
Epoch: 4 Step: 691 loss: 0.5560613430516664 Epoch_Time: 1315.7562897205353
Saved Model Complete
Val: F1 Score:  83.25787587247689 Accuracy:  84.22222222222221 Loss:  0.5245031118392944
Test: F1 Score:  81.663079156

In [ ]:
# num_epochs = 5
# for i in range(num_epochs):
#     start_time = time.time()
#     random = np.random.choice(len(yy_train), size=(len(yy_train)), replace=False)
#     xx_train = xx_train[random]
#     yy_train = yy_train[random]
#     xx_mask_train = xx_mask_train[random]
#     xx_title_train = xx_title_train[random]
#     xx_title_mask_train = xx_title_mask_train[random]                           
#     xx_total_title_train = xx_total_title_train[random]
#     xx_total_title_mask_train = xx_total_title_mask_train[random]
            
#     losses, step = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train
#                         ,xx_total_title_mask_train, train_cond=True)
    
#     print("Epoch:", i+1,"Step:", step, "loss:",losses, "Epoch_Time:",time.time() - start_time)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")

#     val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                                xx_total_title_mask_val, train_cond=False)

#     f1_, accu_ = eval_score(y_val, val_pred)
#     print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
#     test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_mask_test, train_cond=False)

#     f1_, accu_ = eval_score(y_test, test_pred)
#     print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Epoch: 1 Step: 1177 loss: 0.47676222771406174 Epoch_Time: 870.2390775680542
Saved Model Complete
Val: F1 Score:  84.36828141224362 Accuracy:  85.2 Loss:  0.4713286896546682
Test: F1 Score:  84.40428395829146 Accuracy:  85.21052631578947 Loss:  0.464148129287519
Epoch: 2 Step: 1313 loss: 0.46324078563381643 Epoch_Time: 734.300537109375
Saved Model Complete
Val: F1 Score:  80.3817136685002 Accuracy:  82.46666666666667 Loss:  0.497027192513148
Test: F1 Score:  81.68657894576536 Accuracy:  83.55263157894737 Loss:  0.4815854751750043
Epoch: 3 Step: 1449 loss: 0.45736973601229053 Epoch_Time: 730.1822309494019
Saved Model Complete
Val: F1 Score:  85.48934396952018 Accuracy:  86.13333333333333 Loss:  0.4446424126625061
Test: F1 Score:  85.98979735482011 Accuracy:  86.68421052631578 Loss:  0.4219284630135486
Epoch: 4 Step: 1585 loss: 0.4337381093817599 Epoch_Time: 1963.375156879425
Saved Model Complete
Val: F1 Score:  83.65731822543195 Accuracy:  84.13333333333334 Loss:  0.465514479080836
Test:

In [ ]:
# num_epochs = 5
# for i in range(num_epochs):
#     start_time = time.time()
#     random = np.random.choice(len(yy_train), size=(len(yy_train)), replace=False)
#     xx_train = xx_train[random]
#     yy_train = yy_train[random]
#     xx_mask_train = xx_mask_train[random]
#     xx_title_train = xx_title_train[random]
#     xx_title_mask_train = xx_title_mask_train[random]                           
#     xx_total_title_train = xx_total_title_train[random]
#     xx_total_title_mask_train = xx_total_title_mask_train[random]
            
#     losses, step = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train
#                         ,xx_total_title_mask_train, train_cond=True)
    
#     print("Epoch:", i+1,"Step:", step, "loss:",losses, "Epoch_Time:",time.time() - start_time)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")

#     val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                                xx_total_title_mask_val, train_cond=False)

#     f1_, accu_ = eval_score(y_val, val_pred)
#     print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)

Epoch: 1 Step: 1822 loss: 0.42914873122665953 Epoch_Time: 2905.098249197006
Saved Model Complete
Val: F1 Score:  85.41288763719568 Accuracy:  86.1923076923077 Loss:  0.4143218191770407


In [26]:
# test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_mask_test, train_cond=False)

# f1_, accu_ = eval_score(y_test, test_pred)
# print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Test: F1 Score:  84.69131934599442 Accuracy:  85.12121212121212 Loss:  0.4467568763277747


In [24]:
# sum1 = 0
# sum2 = 0
# for i in full_data:
#     if(i[2]==0):
#         sum1 += 1
#     else:
#         sum2 += 1